In [1]:
import os
import random
import numpy as np
from skimage import io, transform
from PIL import Image
import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, LeakyReLU, Activation, Reshape, Input
from keras.layers import Conv2D, Conv2DTranspose, Lambda, Concatenate
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.backend.tensorflow_backend import set_session

K.set_learning_phase(False) 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
def dick(x):
    x = K.expand_dims(x, axis=1)
    x = K.expand_dims(x, axis=2)
    x = K.tile(x, [1, 4, 4, 1])
    return x


class dcgan():
    def __init__(self):
        self.dis_depth = 64
        self.gen_depth = 512
        self.alpha = 0.2
        self.dim = 4
        self.batch = 64
        self.epochs = 300
        
    def getmodelweight(self):
    
        #generator weights
        
        truncate_normal = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02, seed=None)
        
        self.s_dense = Dense(256, kernel_initializer=truncate_normal, name='sd0')
        self.s_leak = LeakyReLU(self.alpha, name='slk')
        
        #4 4 512
        self.g_dense_1 = Dense(self.dim*self.dim*self.gen_depth, kernel_initializer=truncate_normal, name='gd1')
        self.g_conv_1 = Conv2DTranspose(int(self.gen_depth/2), 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc1')
        self.g_conv_2 = Conv2DTranspose(int(self.gen_depth/4), 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc2')
        self.g_conv_3 = Conv2DTranspose(int(self.gen_depth/8), 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc3')
        self.g_conv_4 = Conv2DTranspose(3, 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc4')
        
        self.g_batch_1 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb1')
        self.g_batch_2 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb2')
        self.g_batch_3 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb3')
        self.g_batch_4 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb4')
        
        self.g_resap = Reshape((self.dim, self.dim, self.gen_depth), name='gres')
        self.g_relu = Activation('relu', name='greu')
        self.g_leak = LeakyReLU(self.alpha, name='glk')
        self.g_tanh = Activation('tanh', name='gtan')
        self.g_concat = Concatenate(axis=-1, name='gcon')
        
        #discriminator weights
    
        #32 32 64
        self.d_conv_1 = Conv2D(self.dis_depth*1, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc1') 
        self.d_conv_2 = Conv2D(self.dis_depth*2, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc2')
        self.d_conv_3 = Conv2D(self.dis_depth*4, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc3')
        self.d_conv_4 = Conv2D(self.dis_depth*8, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc4')
        self.d_conv_5 = Conv2D(self.dis_depth*8, 1, strides=1, kernel_initializer=truncate_normal, padding='same', name='dc5')
        
        self.d_leak = LeakyReLU(self.alpha, name='dlk')
        self.d_lamb = Lambda(dick, name='dlm')
        self.d_concat = Concatenate(axis=-1, name='dc')
        self.d_flat = Flatten()
        self.d_dense = Dense(1, kernel_initializer=truncate_normal)
        self.d_sig = Activation('sigmoid')
        
    def getgenerator(self, input_1, input_2):
        
        #generator network
        
        self.gn_concat = self.g_concat([input_1, input_2])
        
        self.gn_dense_1 = self.g_dense_1(self.gn_concat)
        self.gn_batch_1 = self.g_batch_1(self.gn_dense_1)
        self.gn_resap_1 = self.g_resap(self.gn_batch_1)
        
        self.gn_conv_1 = self.g_conv_1(self.gn_resap_1)
        self.gn_batch_2 = self.g_batch_2(self.gn_conv_1)
        self.gn_relu_1 = self.g_relu(self.gn_batch_2)
        
        self.gn_conv_2 = self.g_conv_2(self.gn_relu_1)
        self.gn_batch_3 = self.g_batch_3(self.gn_conv_2)
        self.gn_relu_2 = self.g_relu(self.gn_batch_3)
        
        self.gn_conv_3 = self.g_conv_3(self.gn_relu_2)
        self.gn_batch_4 = self.g_batch_4(self.gn_conv_3)
        self.gn_relu_3 = self.g_relu(self.gn_batch_4)
        
        self.gn_conv_4 = self.g_conv_4(self.gn_relu_3)
        self.gn_tanh = self.g_tanh(self.gn_conv_4)
        
        return self.gn_tanh
        
    def getdiscriminator(self, input_1, input_2):
        
        #discriminator network
        
        #32 32 64
        self.dn_conv_1 = self.d_conv_1(input_1)
        self.dn_leak_1 = self.d_leak(self.dn_conv_1)
        
        #16 16 128
        self.dn_conv_2 = self.d_conv_2(self.dn_leak_1)
        self.dn_leak_2 = self.d_leak(self.dn_conv_2)
        
        #8 8 256
        self.dn_conv_3 = self.d_conv_3(self.dn_leak_2)
        self.dn_leak_3 = self.d_leak(self.dn_conv_3)
        
        #4 4 512
        self.dn_conv_4 = self.d_conv_4(self.dn_leak_3)
        self.dn_leak_4 = self.d_leak(self.dn_conv_4)
        
        self.dn_lamba = self.d_lamb(input_2)
        self.dn_concat = self.d_concat([self.dn_leak_4, self.dn_lamba])
        
        self.dn_conv_5 = self.d_conv_5(self.dn_concat)
        self.dn_leak_5 = self.d_leak(self.dn_conv_5)
        
        self.dn_flat = self.d_flat(self.dn_leak_5)
        
        self.dn_dense = self.d_dense(self.dn_flat)
        return self.dn_dense
        
    def getmodels(self):
        
        input_shape1 = (64, 64, 3)
        input_shape2 = (23,)
        input_shape3 = (100,)

        self.input_1 = Input(shape=input_shape1) #image
        self.input_2 = Input(shape=input_shape2) #seq vec
        self.input_3 = Input(shape=input_shape3) #random noise
        
        self.generator = self.getgenerator(self.input_3, self.input_2)
        self.discriminator = self.getdiscriminator(self.input_1, self.input_2)
        
        self.generatorModel = Model(inputs=[self.input_3, self.input_2], outputs=[self.generator])
        self.discriminatorModel = Model(inputs=[self.input_1, self.input_2], outputs=[self.discriminator])
        
        self.generatorModel.summary()
        self.discriminatorModel.summary()
        
        self.netD_real_input = Input(shape=(64, 64, 3))
        self.netD_wrong_input = Input(shape=(64, 64, 3))
        self.seq2vec = Input(shape=(23,))
        self.wrong_seq2vec = Input(shape=(23,))
        self.noisev = Input(shape=(100,))
        
        self.netD_fake_input = self.generatorModel([self.noisev, self.seq2vec])

#         self.epsilon_input = K.placeholder(shape=(None,1,1,1))
#         self.epsilont_input = K.placeholder(shape=(None,1))
#         self.netD_mixed_input = Input(shape=(64, 64, 3),
#             tensor=self.epsilon_input * self.netD_real_input + (1-self.epsilon_input) * (self.netD_fake_input+self.netD_wrong_input + self.netD_real_input)/3)
        
#         self.mixed_seq2vec = Input(shape=(2400,),
#             tensor=self.epsilont_input * self.seq2vec + (1-self.epsilont_input) * (self.seq2vec + self.seq2vec + self.wrong_seq2vec)/3)
        
        self.d_real = self.discriminatorModel([self.netD_real_input, self.seq2vec])
        self.d_fake = self.discriminatorModel([self.netD_fake_input, self.seq2vec])
        self.d_wrong_image = self.discriminatorModel([self.netD_wrong_input, self.seq2vec])
        self.d_wrong_tag = self.discriminatorModel([self.netD_real_input, self.wrong_seq2vec])
        
        self.d_loss1 = K.mean(K.binary_crossentropy(K.ones_like(self.d_real), K.sigmoid(self.d_real)), axis=-1)
        self.d_loss2 = K.mean(K.binary_crossentropy(K.zeros_like(self.d_fake), K.sigmoid(self.d_fake)), axis=-1)
        self.d_loss3 = K.mean(K.binary_crossentropy(K.zeros_like(self.d_wrong_image), K.sigmoid(self.d_wrong_image)), axis=-1)
        self.d_loss4 = K.mean(K.binary_crossentropy(K.zeros_like(self.d_wrong_tag), K.sigmoid(self.d_wrong_tag)), axis=-1)
#         self.grad_mixed = K.gradients(self.discriminatorModel([self.netD_mixed_input, self.mixed_seq2vec]), [self.netD_mixed_input])[0]
#         self.norm_grad_mixed = K.sqrt(K.sum(K.square(self.grad_mixed), axis=[1,2,3]))
#         self.grad_penalty = K.mean(K.square(self.norm_grad_mixed -1))

        self.d_loss = self.d_loss1 + self.d_loss2 + self.d_loss3 + self.d_loss4


        self.d_training_updates = Adam(lr=1e-4, beta_1=0.5).get_updates(self.discriminatorModel.trainable_weights,[], self.d_loss)
        self.netD_train = K.function([self.netD_real_input, self.netD_wrong_input, self.seq2vec, self.wrong_seq2vec, self.noisev],
                                [self.d_loss],    
                                self.d_training_updates)
        
        self.g_loss = K.mean(K.binary_crossentropy(K.ones_like(self.d_fake), K.sigmoid(self.d_fake)), axis=-1)
        
        self.g_training_updates = Adam(lr=1e-4, beta_1=0.5).get_updates(self.generatorModel.trainable_weights,[], self.g_loss)
        self.netG_train = K.function([self.noisev, self.seq2vec], [self.g_loss], self.g_training_updates)
        
    def loadseq2vec(self):
        self.seq2vec = np.load('condition.npy')
        
    def train(self):
        
        hair = ['orange', 'white', 'aqua', 'gray', 'green', 'red', 'purple', 'pink', 'blue', 'black', 'brown', 'blonde']
        eyes = ['gray', 'black', 'orange', 'pink', 'yellow', 'aqua', 'purple', 'green', 'brown', 'red', 'blue']
        aa = np.zeros([36,23])
        aa[:,hair.index('green')]=1
        aa[:,len(hair)+eyes.index('orange')]=1
        
        bigdata = np.load('../../../GAN/data2.npy')
        
        for k in range(0, self.epochs):
            
            ite=0
            while ite<len(bigdata):
                data = bigdata[ite:(ite+self.batch)]
                vecs = self.seq2vec[ite:(ite+self.batch)]
                wdata = [data[(x+1)%len(data)] for x in range(len(data))]
                wvecs = np.zeros([len(vecs), 23])
                    
                for l in range(len(vecs)):
                    a = random.randint(0, 11)
                    b = random.randint(12, 22)
                    wvecs[l][a] = 1
                    wvecs[l][b] = 1
                
                noise = np.random.normal(0, 1.0, size=[len(data), 100])
                
                for l in range(1):
                    errD,  = self.netD_train([data, wdata, vecs, wvecs, noise])
                    errD = np.mean(errD)

                for l in range(2):
                    errG, = self.netG_train([noise, vecs])
                    errG = np.mean(errG)

                print(errD, errG)
                ite+=self.batch
                    
            print("save")
            noise = np.random.normal(0, 1.0, size=[36, 100])
            images_fake = self.generatorModel.predict([noise, aa])
            width = 6
            new_im = Image.new('RGB', (64*width,64*width))
            for ii in range(width):
                for jj in range(width):
                    index=ii*width+jj
                    image = (images_fake[index]/2+0.5)*255
                    image = image.astype(np.uint8)
                    new_im.paste(Image.fromarray(image,"RGB"), (64*ii,64*jj))
            filename = "images_adam2/fakeFace%d.png"%k
            new_im.save(filename)
            self.generatorModel.save("model_adam2/generator%d.h5"%k)
            self.discriminatorModel.save("model_adam2/discriminator%d.h5"%k)

In [ ]:
dcgan = dcgan()
dcgan.getmodelweight()
dcgan.getmodels()
dcgan.loadseq2vec()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 23)           0                                            
__________________________________________________________________________________________________
gcon (Concatenate)              (None, 123)          0           input_3[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
gd1 (Dense)                     (None, 8192)         1015808     gcon[0][0]                       
__________

In [ ]:
dcgan.train()

(2.7844641, 0.68983179)
(2.7482076, 0.69486547)
(2.7088814, 0.70016932)
(2.6475825, 0.70604885)
(2.592557, 0.71644235)
(2.600286, 0.71810973)
(2.5757182, 0.72416735)
(2.5700421, 0.72450179)
(2.559042, 0.73440462)
(2.5500245, 0.74772036)
(2.5413249, 0.74969041)
(2.5310988, 0.7589283)
(2.5244312, 0.76985902)
(2.5140913, 0.78594142)
(2.5046282, 0.79574859)
(2.4937174, 0.81045175)
(2.478632, 0.82809436)
(2.4639752, 0.8546195)
(2.452261, 0.85793614)
(2.4378035, 0.92388207)
(2.4125156, 0.92688501)
(2.3924165, 1.0139685)
(2.3588386, 1.0832243)
(2.3110929, 1.2159151)
(2.2583818, 1.2698194)
(2.2755666, 1.8663137)
(2.9838572, 0.95279282)
(2.5234704, 0.80684143)
(2.5969951, 0.81972629)
(2.551641, 0.85999405)
(2.4848914, 0.92444366)
(2.4011345, 1.0330155)
(2.3402262, 1.1118073)
(2.3212674, 1.1619686)
(2.3285978, 1.1459808)
(2.3212533, 1.1260333)
(2.3204665, 1.103811)
(2.3224251, 1.1153831)
(2.3106115, 1.1175138)
(2.3089314, 1.1477025)
(2.309696, 1.1516497)
(2.2995615, 1.1774404)
(2.2853117, 1.2675

(2.0758004, 1.541345)
(2.1116076, 1.7899585)
(2.1193006, 1.7732)
(2.0273447, 1.6185102)
(2.0711555, 1.9176069)
(2.0101793, 2.0989518)
(1.9644096, 1.9597569)
(1.9859949, 2.0905299)
(2.0112953, 1.793365)
(2.1197121, 1.5149229)
(2.155695, 1.4821775)
(2.2603889, 1.2142904)
(2.2452433, 1.1763701)
(2.2093835, 1.5781984)
(2.1103549, 1.6538553)
(2.0885143, 1.5275834)
(2.0319946, 1.9624292)
(2.0317976, 1.7361383)
(2.0337083, 1.7542921)
(2.0787282, 1.4682701)
(2.0622606, 1.7663447)
(2.0712209, 1.780067)
(2.0813847, 1.9707164)
(2.0386043, 1.7933042)
(2.057653, 1.6388844)
(2.0475228, 1.7919111)
(2.0719922, 1.8206928)
(2.1560967, 1.7500789)
(2.055249, 1.7755985)
(2.0302196, 2.1709557)
(2.0678973, 2.1832135)
(1.9505181, 1.943491)
(2.0082302, 1.9128131)
(2.0475137, 1.6162922)
(2.2007592, 1.3080086)
(2.1683969, 1.4603844)
(2.1311517, 1.6105487)
(2.0455666, 1.6106901)
(2.0249376, 1.7642758)
(2.0011907, 1.6676419)
(2.0230885, 1.5137106)
(2.0171747, 1.5517859)
(2.116298, 1.5782831)
(2.1237974, 1.4227176)

(2.0742338, 1.7150909)
(2.033761, 1.8443507)
(2.0218415, 1.8696172)
(1.9717698, 1.8346008)
(2.0383675, 1.8425696)
(1.9932942, 1.6567504)
(2.0933857, 1.6866143)
(1.9774659, 1.7008069)
(2.0691814, 1.4893187)
(2.0128453, 1.5145113)
(2.0931432, 1.5366108)
(2.0702333, 1.5415401)
(2.0628831, 1.6948825)
(2.0132332, 1.5646048)
(2.0599308, 1.6019661)
(2.069952, 1.6059947)
(2.0697427, 1.7538519)
(2.0184183, 1.7983382)
(2.0079243, 1.7207673)
(2.0006964, 1.6590126)
(1.9819582, 1.6407132)
(1.9429188, 1.8691541)
(2.054467, 1.3632886)
(2.0474021, 1.5376227)
(2.0035887, 1.7744658)
(1.9801369, 1.6033474)
(1.9748714, 1.5654955)
(2.0772557, 1.4864661)
(2.0269434, 1.6557596)
(2.0094795, 1.7253524)
(1.9978873, 1.6172786)
(2.0194495, 1.5333633)
(2.1505466, 1.611788)
(2.0401249, 1.5673375)
(2.0354958, 1.5331703)
(2.1222494, 1.5430326)
(2.0391028, 1.6573129)
(2.0061331, 1.8569323)
(2.0223122, 1.6439362)
(2.0477974, 1.7973027)
(1.985044, 2.2490132)
(1.9978253, 1.5876771)
(2.0105596, 2.2956815)
(2.0370979, 1.28

(1.9988503, 1.8809428)
(2.0195076, 1.608404)
(2.0747623, 1.531023)
(1.9877439, 1.8714173)
(2.0005655, 1.5557356)
(2.0159779, 1.6289687)
(2.0035698, 1.7142138)
(1.9873921, 1.906948)
(1.8793738, 2.0835526)
(1.9658717, 1.8493069)
(1.9281945, 2.0052314)
(2.0532336, 1.6824079)
(1.9693888, 1.9002818)
(2.0096545, 2.0801044)
(2.1282253, 1.1453001)
(2.1865792, 1.7561078)
(1.9859607, 1.9545168)
(2.0632124, 1.4772333)
(2.0419219, 1.433172)
(1.9620637, 1.8143597)
(1.9549356, 1.8066123)
(1.9250367, 1.8682498)
(2.1129842, 1.3813292)
(2.0583277, 1.5828395)
(2.0609655, 1.726053)
(2.0286651, 1.5746839)
(2.0281889, 1.502975)
(2.0194917, 1.6211184)
(1.9850166, 1.892297)
(1.9650037, 1.6995686)
(1.987407, 1.6487482)
(2.0299323, 1.602582)
(1.9877719, 1.5165451)
(1.9552717, 1.6275997)
(2.0490177, 1.4683304)
(2.0326495, 1.4563422)
(2.0780923, 1.5162337)
(1.9881424, 1.5555094)
(2.0128617, 1.7048482)
(2.006094, 1.6184314)
(1.9976361, 2.0087652)
(1.9339435, 2.1160684)
(2.0048716, 1.7047631)
(2.0425606, 2.0101492

(1.9437931, 1.510464)
(1.9816679, 1.4374313)
(2.0385871, 1.63239)
(1.9999747, 1.5879996)
(1.978119, 1.3752086)
(1.9878931, 1.6265216)
(1.9337285, 1.742826)
(1.9819977, 1.4396988)
(2.0945437, 1.5535023)
(2.111928, 1.3656535)
(1.9641867, 1.6178731)
(2.0591536, 1.5863109)
(2.0156136, 1.4202054)
(1.9672618, 1.7640533)
(2.0042593, 1.7732375)
(1.9722588, 1.5164098)
(1.9917073, 1.3720825)
(1.9624069, 1.5319076)
(2.0230155, 1.6003163)
(1.9301207, 1.7429256)
(1.9173734, 1.8697438)
(1.9875662, 1.728133)
(1.9659194, 1.6386713)
(2.0745416, 1.3596945)
(2.239656, 1.2383134)
(2.0622265, 1.5165062)
(1.9006299, 1.8714881)
(1.8915088, 1.9464548)
(1.8647355, 1.8641806)
(1.9026564, 1.4906095)
(1.9604862, 1.739892)
(2.0335226, 1.5431502)
(2.0165281, 1.4249461)
(1.9853994, 1.6952791)
(2.0160842, 1.7274739)
(1.9737471, 1.7217689)
(1.9174163, 1.754894)
(1.9598722, 1.4621811)
(1.9754426, 1.495159)
(1.9619622, 1.5688372)
(2.0153713, 1.4536053)
(1.93648, 1.3789924)
(1.9623194, 1.6040311)
(1.9692659, 1.6752665)
(

(2.0335779, 1.4184108)
(1.9831376, 1.5323513)
(1.9000932, 1.436197)
(1.9658384, 1.4030615)
(2.0210953, 1.3958374)
(2.0473049, 1.3952031)
(1.9967344, 1.2636199)
(2.0342488, 1.2807124)
(1.9621798, 1.4041321)
(1.8725207, 1.3646147)
(1.9840304, 1.385893)
(2.0599766, 1.5877054)
(1.9763849, 1.526359)
(1.9889573, 1.5077881)
(1.9584587, 1.2772462)
(2.0188556, 1.4587839)
(1.985296, 1.5443348)
(1.9846485, 1.4248438)
(2.0123754, 1.3734384)
(2.0304821, 1.5709903)
(2.0250866, 1.4836295)
(2.1061532, 1.2325649)
(2.0912347, 1.2618148)
(1.9839478, 1.4058833)
(2.0474005, 1.4430729)
(1.9791818, 1.4151987)
(2.007843, 1.403517)
(2.0047381, 1.5401151)
(1.9251418, 1.6379273)
(1.9773545, 1.4480046)
(1.9845824, 1.4285295)
(1.953527, 1.6655166)
(1.9410793, 1.3959939)
(1.9980515, 1.4002048)
(2.0203459, 1.691326)
(1.9189219, 1.6021935)
(1.947525, 1.6596909)
(1.9991457, 1.4183896)
(2.0224123, 1.36408)
(1.9491456, 1.6066593)
(1.9505713, 1.5491092)
(1.8373799, 1.5027785)
(1.9638, 1.6354382)
(1.980749, 1.2268479)
(2.

(1.9572375, 1.27106)
(2.010474, 1.5500277)
(2.0063062, 1.2407479)
(2.0326967, 1.2751374)
(1.9187722, 1.4591315)
(2.0425611, 1.4435)
(1.9385568, 1.4026003)
(1.9491616, 1.3265654)
(2.0670917, 1.418731)
(2.020786, 1.3578328)
(1.9000807, 1.3436079)
(1.985333, 1.309955)
(2.0325472, 1.3913426)
(1.9494079, 1.3335459)
(1.9794321, 1.2923071)
(1.9747863, 1.403857)
(2.0113003, 1.421205)
(2.0110483, 1.2917036)
(1.9372753, 1.3206947)
(1.9690723, 1.4855654)
(1.9049752, 1.4808445)
(1.918533, 1.4834504)
(1.8868425, 1.4394219)
(2.1240883, 1.1786153)
(1.9833918, 1.5357815)
(1.9940439, 1.5934)
(2.006119, 1.329453)
(2.1619191, 1.2169795)
(2.119261, 1.2921963)
(2.0667734, 1.4070946)
(1.9988396, 1.4470097)
(1.9621919, 1.2787378)
(2.0511999, 1.1396192)
(2.0666547, 1.3036622)
(2.0029302, 1.3783305)
(1.9433007, 1.4270788)
(2.0020835, 1.2759981)
(2.0079248, 1.3143743)
(1.9834914, 1.3127592)
(1.9804018, 1.2696956)
(2.0401578, 1.369626)
(1.9926773, 1.4135054)
(1.8897052, 1.3865631)
(1.967857, 1.2827997)
(2.007917